<a href="https://colab.research.google.com/github/MessiNN/chatbot-transformer-Early_Stage-/blob/master/Transformer_Architecture_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


import random
import re
import os
import unicodedata
import itertools
import pandas as pd

from torch.nn.utils.rnn import pad_sequence


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device_cpu = torch.device("cpu")

save_dir = os.path.join("/content/drive/MyDrive", "data", "save")

PAD_token = 0
SOS_token = 1
EOS_token = 2

MAX_LENGTH = 20
MIN_COUNT = 6

Mounted at /content/drive


In [2]:
class Library:
    def __init__(self):
        self.name = "Dataset"
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)


        print(len(keep_words), len(self.word2index), len(keep_words), len(self.word2index))
        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3

        for word in keep_words:
            self.addWord(word)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

def readVocs(datafile):
    pairs = []
    df = pd.read_parquet(datafile)
    questions = df['question'].tolist()
    responses = df['response'].tolist()
    for question, response in zip(questions, responses):
      question = normalizeString(question)
      response = normalizeString(response)
      pair = [question, response]
      pairs.append(pair)
    return pairs

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def loadPrepareData(datafile, save_dir):
    libra = Library()
    pairs = readVocs(datafile)
    pairs = filterPairs(pairs)
    for pair in pairs:
        libra.addSentence(pair[0])
        libra.addSentence(pair[1])
    return libra, pairs


def trimRareWords(libra, pairs, MIN_COUNT):
    libra.trim(MIN_COUNT)
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        for word in input_sentence.split(' '):
            if word not in libra.word2index:
                keep_input = False
                break
        for word in output_sentence.split(' '):
            if word not in libra.word2index:
                keep_output = False
                break

        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs

def indexesFromSentence(libra, sentence):
    return [libra.word2index[word] for word in sentence.split(' ') if word in libra.word2index] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    padded_list = []
    for sequence in l:
        padded_sequence = list(sequence) + [fillvalue] * (MAX_LENGTH - len(sequence))
        padded_list.append(padded_sequence)
    return padded_list

def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar

def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp = inputVar(input_batch, voc)
    output = outputVar(output_batch, voc)
    return inp, output

In [3]:
#Own code implementation (By me)
torch.autograd.set_detect_anomaly(True)

def create_padding_mask(length):
    seq = torch.eq(length, 0)
    return seq.unsqueeze(1).unsqueeze(2)

def create_look_ahead_mask(size):
    mask = (1 - torch.triu(torch.ones(size, size), diagonal=1)).bool()
    return mask

def positions(sequence, size):
    if not isinstance(sequence, int):
        raise ValueError(f"Invalid type for sequence. Expected int, got: {type(sequence)}")

    if not isinstance(size, int):
        raise ValueError(f"Invalid type for size. Expected int, got: {type(size)}")

    if sequence <= 0:
        raise ValueError("Sequence length must be positive.")

    if size <= 0:
        raise ValueError("Size of positional encoding must be positive.")

    batch_size = sequence
    pos = torch.arange(batch_size).float().unsqueeze(1)
    i = torch.arange(size).float().unsqueeze(0)
    angles = pos / 10000 ** (2 * (i // 2) / size)
    angles = angles.reshape(batch_size, size)

    pe = torch.zeros(batch_size, size)
    pe[:, 0::2] = torch.sin(angles[:, 0::2])
    pe[:, 1::2] = torch.cos(angles[:, 1::2])

    return pe

def accuracy(y_true, y_pred):
    y_pred_argmax = torch.argmax(y_pred, dim=-1)
    correct = (y_pred_argmax == y_true).float().sum()
    total = y_true.numel()
    return correct / total

class CustomSchedule(LambdaLR):
    def __init__(self, optimizer, d_model, warmup_steps=4000):
        self.d_model = torch.tensor(d_model, dtype=torch.float32)
        self.warmup_steps = warmup_steps
        super(CustomSchedule, self).__init__(optimizer, self.lr_lambda)

    def lr_lambda(self, step):
        step_float = torch.tensor(step, dtype=torch.float32)
        arg1 = torch.rsqrt(step_float)
        arg2 = step_float * (self.warmup_steps**-1.5)
        return (torch.rsqrt(self.d_model) * torch.minimum(arg1, arg2))

class Normalize(nn.Module):
  def __init__(self, scale: float, shift:float,):
    super(Normalize, self).__init__()
    self.scale = scale
    self.shift = shift

  def forward(self, x):
      mean = torch.mean(x)
      deviation = torch.std(x)
      x = (x - mean) / deviation
      x = x * self.scale
      x = x + self.shift
      return x


#---- Self Made Multi Head Attention Implementation ----
class Multi_Head_Attention(nn.Module):
    def __init__(self, embedding_size, head_num, batch_size, learning_rate):
        super(Multi_Head_Attention, self).__init__()

        self.embedding_size = embedding_size
        self.num_heads = head_num
        self.learning_rate = learning_rate
        self.head_size = embedding_size // self.num_heads

        self.head_size = int(self.head_size)

        self.w_q = nn.Linear(embedding_size, embedding_size, bias=False)
        self.w_k = nn.Linear(embedding_size, embedding_size, bias=False)
        self.w_v = nn.Linear(embedding_size, embedding_size, bias=False)

        self.w_o = nn.Linear(embedding_size, embedding_size, bias=False)

    def scaled_dot_product_attention(self, x, blank, mask=None):
        batch, length, size = x.shape
        if blank is not None:
            b_batch, b_length, b_size = blank.shape
            q = self.w_q(blank).view(b_batch, b_length, self.num_heads, self.head_size).permute(0,2,1,3)
        else:
            q = self.w_q(x).view(batch, length, self.num_heads, self.head_size).permute(0,2,1,3)
        k = self.w_k(x).view(batch, length, self.num_heads, self.head_size).permute(0,2,1,3)
        v = self.w_v(x).view(batch, length, self.num_heads, self.head_size)

        q = q.permute(0, 2, 3, 1)
        k = k.permute(0, 2, 1, 3)

        distribution = torch.matmul(q, k)/ torch.sqrt(torch.tensor(self.embedding_size))

        if mask is not None:
            distribution.masked_fill(mask == 0, float('-inf'))

        v = v.transpose(2, 3)
        distribution = F.softmax(distribution, dim=-1)
        weight = torch.matmul(distribution, v)
        return weight

    def forward(self, x, blank=None, mask=None ):
        #shape: batch / length /embedding_size
        batch, length, size = x.shape

        product = self.scaled_dot_product_attention(x, blank, mask)

        product = product.permute(0,2,1,3)

        concat = product.reshape(batch, length, self.embedding_size)

        out = self.w_o(concat)

        return out

#----Self Made Layer Encoder Implementation----
class Encoder_Layer(nn.Module):
    def __init__(self, embedding_size, head_num, batch_size, dropout, learning_rate):
        super(Encoder_Layer, self).__init__()

        self.embedding_size = embedding_size

        self.fc1 = nn.Linear(embedding_size, embedding_size, bias=False)
        self.fc2 = nn.Linear(embedding_size, embedding_size, bias=False)

        self.attention = Multi_Head_Attention(embedding_size, head_num, batch_size, learning_rate)

        self.norm = Normalize(0.4, 0.4)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn = self.attention(x, None, None)

        attn_d = self.dropout(attn)
        attn_dn = self.norm(x + attn_d)

        attn_lin1 = self.fc1(attn_dn)
        attn_rel = F.relu(attn_lin1)

        attn_lin2 = self.fc2(attn_rel)
        attn_d = self.dropout(attn_lin2)
        out = self.norm(attn_dn + attn_d)

        return out

#----Self Made Encoder Implementation----
class Encoder(nn.Module):
    def __init__(self, embedding, embedding_size, head_num, batch_size, dropout, learning_rate, n_layers=2):
        super(Encoder, self).__init__()

        self.num_layers = n_layers
        self.encoder_layers = nn.ModuleList([Encoder_Layer(embedding_size, head_num, batch_size, dropout, learning_rate) for _ in range(n_layers)])

        self.embedding = embedding
        self.embedding_size = embedding_size

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # shape: batch / length / embedding_size
        batch, length, embedding_size = x.shape


        pos = positions(length, self.embedding_size).to(device)
        input_pos = pos + x

        input_d = self.dropout(input_pos)#.to(device)

        for encoder_layer in self.encoder_layers:
            output = encoder_layer(input_d, mask)

        return output

#----Self Made Layer Decoder Implementation----
class Decoder_Layer(nn.Module):
    def __init__(self, embedding_size, head_num, batch_size, dropout, learning_rate):
        super(Decoder_Layer, self).__init__()

        self.embedding_size = embedding_size

        self.fc1 = nn.Linear(embedding_size, embedding_size, bias=False)
        self.fc2 = nn.Linear(embedding_size, embedding_size, bias=False)

        self.attention = Multi_Head_Attention(embedding_size, head_num, batch_size, learning_rate)

        self.norm = Normalize(0.4, 0.4)
        self.dropout = nn.Dropout(dropout)

    def forward(self, enc_o, blank, ahead, mask):
        attn1 = self.attention(enc_o, None, mask)
        attn_n = self.norm(attn1 + enc_o)

        attn2 = self.attention(enc_o, blank, ahead)
        attn_d = self.dropout(attn2)
        attn_n2d = self.norm(attn_n + attn_d)

        attn_lin1 = self.fc1(attn_n2d)
        attn_rel = F.relu(attn_lin1)

        attn_lin2 = self.fc2(attn_rel)
        attn_d = self.dropout(attn_lin2)
        output = self.norm(attn_d + attn_n2d)

        return output

#----Self Made Decoder Implementation----
class Decoder(nn.Module):
    def __init__(self, embedding, embedding_size, head_num, batch_size, dropout, learning_rate, n_layers):
        super(Decoder, self).__init__()
        self.embedding = embedding
        self.embedding_size = embedding_size
        self.num_layers = n_layers

        self.decoder_layers = nn.ModuleList([Decoder_Layer(embedding_size, head_num, batch_size, dropout, learning_rate) for _ in range(n_layers)])

        self.dropout = nn.Dropout(dropout)

    def forward(self, decoder_input, enc_output, ahead=None, mask=None):
        # shape: batch/ length /embedding_size (GENERAL SHAPE)

        embedded = self.embedding(decoder_input.t()).to(device)

        batch, length, embedding_size = enc_output.shape

        batch, length, embedding_size = embedded.shape

        pos = positions(length, self.embedding_size).to(device)
        input_pos = pos + enc_output

        input_d = self.dropout(input_pos)#.to(device)
        for decoder_layer in self.decoder_layers:
            output = decoder_layer(input_d, embedded, ahead, mask)

        return output

class Transformer(nn.Module):
    def __init__(self, embedding_size, head_num, batch_size, dropout, learning_rate, decoder_learning_ratio,
                 encoder_n_layers, decoder_n_layers, vocab_size, libra, task='train', loadFilename=None):
        super(Transformer, self).__init__()

        self.embedding_size = embedding_size
        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.num_heads = head_num
        self.task = task

        self.fc = nn.Linear(embedding_size, vocab_size, bias=False)
        self.embedding = nn.Embedding(vocab_size, embedding_size)

        self.decoder = Decoder(self.embedding, embedding_size, head_num, batch_size, dropout, learning_rate, encoder_n_layers)
        self.encoder = Encoder(self.embedding, embedding_size, head_num, batch_size, dropout, learning_rate, decoder_n_layers)

        if loadFilename:
            print("Set to: 'trained model'")
            self.checkpoint = torch.load(loadFilename, map_location=device)
            encoder_sd = self.checkpoint['en']
            decoder_sd = self.checkpoint['de']
            encoder_optimizer_sd = self.checkpoint['en_opt']
            decoder_optimizer_sd = self.checkpoint['de_opt']
            embedding_sd = self.checkpoint['embedding']
            libra.__dict__ = self.checkpoint['voc_dict']
            print("Loss: ",self.checkpoint["loss"])
            print("Time: ",self.checkpoint["time"])

        else:
            print("Set to: 'new model'")

        if loadFilename:
            self.embedding.load_state_dict(embedding_sd)

        if loadFilename:
            self.encoder.load_state_dict(encoder_sd)
            self.decoder.load_state_dict(decoder_sd)

        self.embedding = self.embedding.to(device)
        self.encoder = self.encoder.to(device)
        self.decoder = self.decoder.to(device)

        if task == "train":
            self.encoder.train()
            self.decoder.train()
        else:
            self.encoder.eval()
            self.decoder.eval()

        self.decoder_optimizer = optim.Adam(self.decoder.parameters(), lr=learning_rate, betas=(0.9, 0.98), eps=1e-9)
        self.encoder_optimizer = optim.Adam(self.encoder.parameters(), lr=learning_rate * decoder_learning_ratio, betas=(0.9, 0.98), eps=1e-9)

        self.decoder_scheduler = CustomSchedule(self.decoder_optimizer, embedding_size)
        self.encoder_scheduler = CustomSchedule(self.encoder_optimizer, embedding_size)

        if loadFilename:
            self.encoder_optimizer.load_state_dict(encoder_optimizer_sd)
            self.decoder_optimizer.load_state_dict(decoder_optimizer_sd)

    def forward(self, input_variable):
        # shape:  batch / length / embedding_size
        if len(input_variable.shape) == 3:
          batch, length, d_model = input_variable.size()
        else:
          input_variable = self.embedding(input_variable)
          batch, length, d_model = input_variable.size()

        enc_padding_mask = create_padding_mask(torch.LongTensor(length)).to(device)
        dec_padding_mask = create_padding_mask(torch.LongTensor(length)).to(device)
        look_ahead_mask  = create_look_ahead_mask(self.embedding_size // self.num_heads).to(device)

        decoder_input = torch.LongTensor([[SOS_token for _ in range(batch)]]).to(device)

        enc_output = self.encoder(input_variable, enc_padding_mask)
        dec_output = self.decoder(decoder_input, enc_output, look_ahead_mask, dec_padding_mask)

        output = self.fc(dec_output)
        decoder_output_probs = F.softmax(output, dim=-1)

        return decoder_output_probs

In [4]:
def train(input_variable, target_variable, vocab_size, model, clip, max_length=MAX_LENGTH):

    model.encoder_optimizer.zero_grad()
    model.decoder_optimizer.zero_grad()

    t_loss = 0
    t_accuracy = 0
    n_totals = 0

    for t in range(max_length):
        decoder_output = model(input_variable)
        loss = F.cross_entropy(decoder_output.view(-1, vocab_size), target_variable.reshape(-1), ignore_index=0)
        mask = (target_variable != 0).float()
        loss = (loss * mask).mean()
        accuracy_v = accuracy(target_variable, decoder_output)
        t_loss += loss
        t_accuracy += accuracy_v.item()
        n_totals += 1

    loss.backward()

    model.encoder_optimizer.step()
    model.decoder_optimizer.step()

    model.encoder_scheduler.step()
    model.decoder_scheduler.step()

    _ = nn.utils.clip_grad_norm_(model.encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(model.decoder.parameters(), clip)

    return t_loss / n_totals, t_accuracy / n_totals

In [5]:
def trainIters(model_name, libra, pairs, save_dir, n_iteration, batch_size,
               print_every, save_every, clip, loadFilename, vocab_size, model):

    print("Creating the training batches...")
    training_pairs = [batch2TrainData(libra, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    start_iteration = 1
    print_loss = 0
    print_count = 0
    old = 0
    tries = 0

    if loadFilename:
        tries = model.checkpoint['time']

    print("Initializing Training...")
    print()
    for iteration in range(start_iteration, n_iteration + 1):
        training_pair = training_pairs[iteration - 1]

        input_variable, target_variable = training_pair

        input_variable = input_variable.to(device)
        target_variable = target_variable.to(device)
        try:

          input_variable = model.embedding(input_variable).to(device)

          loss, accuracy = train(input_variable, target_variable, vocab_size, model, clip)

          if iteration % print_every == 0:
              print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}; Average accuracy: {:.4f}".format(iteration, iteration / n_iteration * 100, loss, accuracy))


          if (iteration % save_every == 0):
                      tries += save_every
                      directory = os.path.join(save_dir, model_name, '{}-{}_{}'.format(model.embedding_size, model.num_heads, model.vocab_size))
                      if not os.path.exists(directory):
                          os.makedirs(directory)
                      torch.save({
                          'iteration': iteration,
                          'time': tries,
                          'mo':model,
                          'en': model.encoder.state_dict(),
                          'de': model.decoder.state_dict(),
                          'en_opt': model.encoder_optimizer.state_dict(),
                          'de_opt': model.decoder_optimizer.state_dict(),
                          'loss': loss,
                          'voc_dict': libra.__dict__,
                          'embedding': model.embedding.state_dict()
                      }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))
        except:
              tries += save_every
              directory = os.path.join(save_dir, "Emergency Save", '{}-{}_{}'.format(model.embedding_size, model.num_heads, model.vocab_size))
              if not os.path.exists(directory):
                  os.makedirs(directory)
              torch.save({
                  'iteration': iteration,
                  'time': tries,
                  'mo':model,
                  'en': model.encoder.state_dict(),
                  'de': model.decoder.state_dict(),
                  'en_opt': model.encoder_optimizer.state_dict(),
                  'de_opt': model.decoder_optimizer.state_dict(),
                  'loss': loss,
                  'voc_dict': libra.__dict__,
                  'embedding': model.embedding.state_dict()
              }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [6]:
def evaluate(sentence, model, libra, embedding_size):
    # Tokenize the sentence
    tokenized_sentence = [SOS_token] + [libra.word2index[word] for word in sentence.split()] + [EOS_token]
    # Convert to tensor and add batch dimension
    sentence_tensor = torch.tensor(tokenized_sentence, dtype=torch.long).unsqueeze(0)

    # Initialize output tensor with start token
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])

    for i in range(MAX_LENGTH):
        with torch.no_grad():
            predictions = model(sentence_tensor)

        # Select the last word from the seq_len dimension
        predictions = predictions[:, -1:, :]
        predicted_id = torch.argmax(predictions, axis=-1)

        # Return the result if the predicted_id is equal to the end token
        if predicted_id.item() == EOS_token:
            break

        # Concatenate the predicted_id to the output
        output = torch.cat([decoder_input, predicted_id], axis=-1)

    return output.squeeze(0)


def predict(input_sentence, model, libra, embedding_size):
    prediction = evaluate(input_sentence, model, libra, embedding_size)
    predicted_sentence = [libra.index2word[index.item()] for index in prediction if index.item() < libra.num_words]
    return predicted_sentence


def evaluateInput(model, libra, embedding_size):
    input_sentence = ''
    while(1):
        try:
            input_sentence = input('User > ')
            if input_sentence == 'q' or input_sentence == 'quit': break
            input_sentence = normalizeString(input_sentence)
            output_words = predict(input_sentence, model, libra, embedding_size)
            print('Cleopatra:', ' '.join(output_words))
        except KeyError:
            print("Error: Encountered unknown word.")

In [7]:
parquet_path = "/content/drive/MyDrive/movie-corpus/movie-corpus/0000.parquet"
libra, pairs = loadPrepareData(parquet_path, save_dir)
pairs = trimRareWords(libra, pairs, MIN_COUNT)

1365 12010 1365 12010
keep_words 1365 / 12010 = 0.1137
Trimmed from 3165 pairs to 184, 0.0581 of total


In [ ]:
model_name = 'Cleopatra_model'

encoder_n_layers = 4
decoder_n_layers = 6
embedding_size = 744
head_num = 8

if embedding_size % head_num != 0:
    raise ValueError("embedding_size / head_num must result in an integer")

dropout = 0.2
batch_size = 20
decoder_learning_ratio = 0.80
learning_rate = 0.001
vocab_size = libra.num_words
task = "train"

start_model = "no"
loadFilename = None if start_model == "no" else "/content/drive/MyDrive/data/save/Cleopatra_model/512-8_1368/1000_checkpoint.tar"

model = Transformer(embedding_size, head_num, batch_size, dropout, learning_rate, decoder_learning_ratio,
                 encoder_n_layers, decoder_n_layers, vocab_size, libra, task, loadFilename)

model = model.to(device)

clip = 50.0
n_iteration = 10000
print_every = 100
save_every = 1000

if task == "train":
    trainIters(model_name, libra, pairs, save_dir, n_iteration, batch_size,
               print_every, save_every, clip, loadFilename, vocab_size, model)

if task == "test":
    evaluateInput(model, libra, embedding_size)

Set to: 'new model'
Creating the training batches...
Initializing Training...

Iteration: 100; Percent complete: 1.0%; Average loss: 3.0148; Average accuracy: 0.0001
Iteration: 200; Percent complete: 2.0%; Average loss: 3.6286; Average accuracy: 0.0000
